To connect to an Azure workspace replace the following variables with your own values.

In [ ]:
subscription_id = 'xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx'
resource_group = 'myresourcegroup'
workspace_name = 'myworkspace'
location = 'westus'


Import the Q# package. Set the target profile to be the base profile.

In [1]:
import qsharp

qsharp.init(target_profile=qsharp.TargetProfile.Base)


Q# initialized with configuration: {'targetProfile': 'base', 'languageFeatures': None, 'manifest': None}

The following will generate a compiler error because it uses a feature unsupported by the base profile.

In [2]:
%%qsharp

operation NotAllowed() : Unit {
    use q = Qubit();
    let result = M(q);
    if (result == Zero) {
        Message("The result is Zero");
    }
    Reset(q);
}


QSharpCellError: [31mQsc.CapabilitiesCk.UseOfDynamicBool[0m

  [31m×[0m cannot use a dynamic bool value
   ╭─[[36;1;4mline_0[0m:4:1]
 [2m4[0m │     let result = M(q);
 [2m5[0m │     if (result == Zero) {
   · [35;1m        ──────────────[0m
 [2m6[0m │         Message("The result is Zero");
   ╰────
[36m  help: [0musing a bool value that depends on a measurement result is not
        supported by the current target


Define some more Q# operations. This should compile without error.

In [3]:
%%qsharp

operation Random() : Result {
    use q = Qubit();
    H(q);
    let result = M(q);
    Reset(q);
    return result
}

operation RandomNBits(N: Int): Result[] {
    mutable results = [];
    for i in 0 .. N - 1 {
        let r = Random();
        set results += [r];
    }
    return results
}


Submit to Azure Quantum with a custom entry expression.

Make sure the `azure-quantum` package is installed.

In [ ]:
%pip install azure-quantum


In [4]:
import azure.quantum

operation = qsharp.compile("RandomNBits(4)")

workspace = azure.quantum.Workspace(
    subscription_id=subscription_id,
    resource_group=resource_group,
    name=workspace_name,
    location=location,
)
target = workspace.get_targets("rigetti.sim.qvm")
job = target.submit(operation, "my-azure-quantum-job", shots=100)
job.get_results()


.........

{'[0, 1, 1, 1]': 0.08,
 '[1, 1, 0, 0]': 0.1,
 '[0, 0, 1, 0]': 0.04,
 '[0, 1, 0, 0]': 0.05,
 '[1, 0, 1, 0]': 0.05,
 '[1, 0, 0, 0]': 0.07,
 '[0, 1, 0, 1]': 0.07,
 '[1, 0, 1, 1]': 0.07,
 '[0, 0, 0, 0]': 0.08,
 '[1, 1, 1, 0]': 0.05,
 '[0, 0, 0, 1]': 0.1,
 '[0, 0, 1, 1]': 0.04,
 '[0, 1, 1, 0]': 0.09,
 '[1, 0, 0, 1]': 0.04,
 '[1, 1, 1, 1]': 0.05,
 '[1, 1, 0, 1]': 0.02}